### 데이터셋 설명
- 팀스파르타가 지난해 처음 선보인 IT인재 채용 플랫폼, **인텔리픽**은 기업에게는 뛰어난 인재들을 채용 가능한 공간으로, 지원자 입장에서는 믿을 수 있는 기업을 선택할 수 있는 공간으로 자리매김 했습니다. 이러한 채용 플랫폼 **인텔리픽**에서 얻어진 데이터를 활용하여 분석을 진행하는 것이 여러분들의 임무입니다 🙂

- **데이터를 구성하고 있는 column에 대한 설명**
    - **`createdat`**: 이력이 생성된 날짜 정보
    - **`userid`**: 사용자를 구분하는 고유 식별자.
    - **`type`**: 해당 이력이 이력서 첨삭(`resume`)인지, 면접 코칭(`interview`)인지를 나타냄.
    - **`result`**: 각 서비스 타입(`type`)에서의 결과(`FAIL`, `PASS`, `CANCEL`).
    - **`course`**: 사용자가 참여하고 있는 교육 과정(`NBCamp`, `Hanghae`).
    - **`status`**: 사용자 최종 상태(`취업 준비중`, `최종합격`, `취업 보류`).

In [1]:
import pandas as pd

df = pd.read_csv("데이터분석온보딩과제.csv")
df

,createdat,userid,type,result,course,status
0,23/07/03 1:31,12a11b388392a1,resume,FAIL,NBCamp,취업 준비중
1,23/07/03 11:27,8399fb63a855cb,resume,FAIL,NBCamp,취업 준비중
2,23/07/03 11:33,223856945edb24,interview,PASS,Hanghae,취업 준비중
3,23/07/03 15:09,94a4d087e77881,resume,PASS,NBCamp,취업 준비중
4,23/07/03 22:06,60becf747f196f,resume,FAIL,Hanghae,취업 준비중
...,...,...,...,...,...,...
2098,23/12/29 22:25,348ec7700e02bc,resume,FAIL,NBCamp,취업 준비중
2099,23/12/30 14:18,82ae0852b17ba9,resume,PASS,NBCamp,취업 준비중
2100,23/12/31 3:04,37b35594ebca42,resume,FAIL,NBCamp,취업 준비중
2101,23/12/31 3:10,37b35594ebca42,resume,FAIL,NBCamp,취업 준비중


#### 1. 주어진 데이터 셋에서, 각각 아래 질문에 해당하는 답을 채워주세요. (SQL / python 코드 등 활용)
    1. 인텔리픽의 취업코칭 서비스를 한 번 이상 활용한 인원은 몇명인가요? 
    2. 해당 데이터에서 최종합격한 인원은 몇명이고, 최종 합격 그룹의 이력서 진단 / 면접 코칭 신청 평균 횟수는 몇 회 인가요?
    3. 이력서 진단 + 면접 코칭을 가장 많이 활용한 인원의 각 횟수는 몇번이며, 해당 인원의 id 값은 무엇인가요?

In [5]:
# 1번 문항
num_users_used_service = df["userid"].nunique()
num_users_used_service

790

In [9]:
# 2번 문항
final_pass_users = df[df["status"] == "최종합격"]["userid"].nunique()
final_pass_group = df[df["status"] == "최종합격"]
avg_resume_interview_count = final_pass_group.groupby("userid")["type"].count().mean()

print(final_pass_users, avg_resume_interview_count)

241 3.975103734439834


In [35]:
# 3번 문항
user_usage_counts = df.groupby("userid")["type"].count()
max_usage_count = user_usage_counts.max()
most_active_user = user_usage_counts.idxmax()

resume_count = df[(df["userid"]==most_active_user) & (df["type"]=="resume")].shape[0]
interview_count = df[(df["userid"]==most_active_user) & (df["type"]=="interview")].shape[0]

print(max_usage_count, most_active_user, resume_count, interview_count)

11 1133a29433dd78 8 3


#### 2. 주어진 데이터 셋에서, 각각 아래 질문에 해당하는 답을 작성하고 근거가 필요한 문제의 경우 답을 도출한 근거를 설명해 주세요.
    1. 첫 취업코칭 신청 이후, 다음 코칭을 받기 까지의 평균 어느정도의 기간이 걸렸나요? 어떤 방식으로 평균 기간을 도출했나요? (한번만 코칭을 받은 사람은 제외)
    2. 인텔리픽 서비스 이용자 중, course별 최종 합격인원은 각각 몇 명, 몇 퍼센트인가요?
    3. 취업 코칭 이후 평균 대기기간이 실제 합격 여부에 통계적으로 유의미한 영향을 주는지 아닌지 판단하고 어떤 근거로 그러한 판단을 했는지 작성하세요.

In [ ]:
# 2-3. 취업 코칭 이후 평균 대기기간이 합격 여부에 미치는 영향 검정 (통계적 검정)
import scipy.stats as stats

# 최종합격 / 취업 준비중 그룹 나누기
final_pass_group = df[df["status"] == "최종합격"].groupby("userid")["createdat"].min()
waiting_group = df[df["status"] == "취업 준비중"].groupby("userid")["createdat"].min()

# 첫 코칭 이후 대기 기간 계산
final_pass_wait = (final_pass_group - df.groupby("userid")["createdat"].min()).dt.days
waiting_group_wait = (waiting_group - df.groupby("userid")["createdat"].min()).dt.days
print(final_pass_wait, waiting_group_wait)
# 두 그룹 간 t-test 수행 (귀무가설: 대기기간 차이가 없다)
t_stat, p_value = stats.ttest_ind(final_pass_wait.dropna(), waiting_group_wait.dropna(), equal_var=False)

(t_stat, p_value)

userid
100547742f167a    0.0
1033f630772c0b    NaN
103aa998952bc3    0.0
10583770ca51be    NaN
106f9f885fda28    0.0
                 ... 
99b98440e1c691    NaN
99c7f086a6e73b    0.0
99e05b79363fc6    NaN
99e0d646fa2280    0.0
99e5e3390b1ab4    NaN
Name: createdat, Length: 790, dtype: float64 userid
100547742f167a    NaN
1033f630772c0b    NaN
103aa998952bc3    NaN
10583770ca51be    0.0
106f9f885fda28    NaN
                 ... 
99b98440e1c691    0.0
99c7f086a6e73b    NaN
99e05b79363fc6    0.0
99e0d646fa2280    NaN
99e5e3390b1ab4    0.0
Name: createdat, Length: 790, dtype: float64


(np.float64(nan), np.float64(nan))

In [ ]:
# 1번 문항

# 날짜 데이터 변환
df["createdat"] = pd.to_datetime(df["createdat"], format="%Y-%m-%d %H:%M:%S")

# 각 사용자의 첫 번째와 두 번째 취업코칭 신청 날짜 찾기
df_sorted = df.sort_values(by=["userid", "createdat"])  # userid별로 시간순 정렬

# 첫 번째와 두 번째 신청 날짜를 찾기 위해 그룹화 후 rank(method="first") 사용
df_sorted["rank"] = df_sorted.groupby("userid")["createdat"].rank(method="first")

# 첫 번째 신청 날짜
first_coaching_dates = df_sorted[df_sorted["rank"] == 1].set_index("userid")["createdat"]

# 두 번째 신청 날짜
second_coaching_dates = df_sorted[df_sorted["rank"] == 2].set_index("userid")["createdat"]

# 데이터프레임으로 정리
user_coaching_dates = pd.DataFrame({
    "first_coaching": first_coaching_dates,
    "second_coaching": second_coaching_dates
})

# 한 번만 받은 사람 제외 (NaT 값 제거거)
valid_users = user_coaching_dates.dropna()

# 첫 번째와 두 번째 코칭 신청 사이의 기간 계산
valid_users["duration"] = (valid_users["second_coaching"] - valid_users["first_coaching"]).dt.days

# 평균 기간 계산
average_duration = valid_users["duration"].mean()
print(average_duration)

userid
100547742f167a   2023-09-26 00:52:00
1033f630772c0b   2023-12-11 15:08:00
103aa998952bc3   2023-09-26 02:06:00
106f9f885fda28   2023-07-06 11:35:00
10759e844d7663   2023-09-01 16:19:00
                         ...        
99b98440e1c691   2023-11-13 23:06:00
99c7f086a6e73b   2023-07-15 01:29:00
99e05b79363fc6   2023-12-07 14:47:00
99e0d646fa2280   2023-08-10 00:03:00
99e5e3390b1ab4   2023-09-08 11:55:00
Name: second_coaching, Length: 551, dtype: datetime64[ns]
7.054446460980036


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_24368\3793851042.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_users["duration"] = (valid_users["second_coaching"] - valid_users["first_coaching"]).dt.days


In [99]:
# 2번 문항

# course별 최종 합격 인원 및 전체 인원
course_final_pass = df[df["status"] == "최종합격"].groupby("course")["userid"].nunique()
course_total_users = df.groupby("course")["userid"].nunique()

# 최종 합격 비율 계산
course_final_pass_ratio = (course_final_pass / course_total_users) * 100

# course별 최종 합격 인원 및 최종 합격 비율
course_final = pd.merge(course_final_pass, course_final_pass_ratio, left_on="course", right_on="course", how="left")
course_final

,userid_x,userid_y
course,,
Hanghae,106,35.099338
NBCamp,135,27.663934


In [ ]:
# 3번 문항

